In [1]:
import kotlin.io.path.Path
import kotlin.io.path.readLines

val input = Path("day10.txt").readLines()

typealias Sketch = List<String>
data class Position(val row: Int, val column: Int)

fun Position.top() = Position(row - 1, column)
fun Position.bottom() = Position(row + 1, column)
fun Position.left() = Position(row, column - 1)
fun Position.right() = Position(row, column + 1)

# Part 1

In [2]:
fun getResult(sketch: Sketch): Int {
    val path = getLoop(sketch)
    return path.size / 2
}

fun getLoop(sketch: Sketch): List<Position> {
    val animalPosition = sketch.mapIndexed { i, row -> Position(i, row.indexOf('S')) }.find { p -> p.column >= 0 }!!
    return getPath(animalPosition, sketch)
}

fun getPath(startingPosition: Position, sketch: Sketch): List<Position> {
    val (firstConnectingPipe, secondConnectingPipe) = getConnectingPipes(sketch, startingPosition)
    val path = mutableListOf<Position>(startingPosition, firstConnectingPipe)
    while (path.last() != secondConnectingPipe) {
        val connectingPipes = getConnectingPipes(sketch, path.last())
        val nextPipe = connectingPipes.find { it != path.get(path.size - 2) }!!
        path.add(nextPipe)
    }
    return path
}

fun getTile(sketch: Sketch, position: Position) = sketch.getOrNull(position.row)?.getOrNull(position.column)

fun getConnectingPipes(sketch: Sketch, position: Position): List<Position> {
    val positions = listOf(position.top(), position.bottom(), position.left(), position.right())
    return positions.filter { areConnected(sketch, position, it) }
}

fun areConnected(sketch: Sketch, position1: Position, position2: Position): Boolean {
    val tile1 = getTile(sketch, position1)
    val tile2 = getTile(sketch, position2)

    if (tile1 == null || tile2 == null) {
        return false
    }

    if (position1.top() == position2) {
        return listOf('S', '|', 'L', 'J').contains(tile1) && listOf('S', '|', '7', 'F').contains(tile2)
    }
    if (position1.bottom() == position2) {
        return listOf('S', '|', '7', 'F').contains(tile1) && listOf('S', '|', 'L', 'J').contains(tile2)
    }
    if (position1.left() == position2) {
        return listOf('S', '-', '7', 'J').contains(tile1) && listOf('S', '-', 'F', 'L').contains(tile2)
    }
    if (position1.right() == position2) {
        return listOf('S', '-', 'F', 'L').contains(tile1) && listOf('S', '-', '7', 'J').contains(tile2)
    }

    return false
}

## Example 1

In [3]:
val example = listOf(
    ".....",
    ".S-7.",
    ".|.|.",
    ".L-J.",
    ".....",
)
getResult(example)


4

## Example 2

In [4]:
val example2 = listOf(
    "..F7.",
    ".FJ|.",
    "SJ.L7",
    "|F--J",
    "LJ..."
)
getResult(example2)

8

## Solution

In [5]:
getResult(input)

7030

# Part 2

In [6]:
fun getInsideCount(sketch: Sketch): Int {
    val path = getLoop(sketch)
    val allPositions = sketch.flatMapIndexed { i, row -> row.indices.map { Position(i, it) } }
    val floorPositions = allPositions.toSet().minus(path.toSet())
    return floorPositions.count { isInside(path, it, sketch) }
}

fun getReplacedTile(sketch: Sketch, position: Position, path: List<Position>): Char {
    val tile = getTile(sketch, position)!!
    if (tile != 'S') {
        return tile
    } else {
        val neighbors = setOf(path[1], path.last())
        val hasTopNeighbor = neighbors.any { it == position.top() }
        val hasBottomNeighbor = neighbors.any { it == position.bottom() }
        val hasLeftNeighbor = neighbors.any { it == position.left() }
        val hasRightNeighbor = neighbors.any { it == position.right() }
        if (hasTopNeighbor && hasBottomNeighbor) {
            return '|'
        }
        if (hasTopNeighbor && hasLeftNeighbor) {
            return 'J'
        }
        if (hasTopNeighbor && hasRightNeighbor) {
            return 'L'
        }
        if (hasBottomNeighbor && hasLeftNeighbor) {
            return '7'
        }
        if (hasBottomNeighbor && hasRightNeighbor) {
            return 'F'
        }
        if (hasLeftNeighbor && hasRightNeighbor) {
            return '-'
        }

        throw Exception("invalid")
    }
}

fun isInside(path: List<Position>, position: Position, sketch: Sketch): Boolean {
    val leftPositions = path.filter { it.row == position.row && it.column < position.column }
    
    val countCrossings = leftPositions.count {
        val tile = getReplacedTile(sketch, it, path)
        "|JL".contains(tile)
    }
    
    return countCrossings % 2 != 0
}

## Example 1

In [7]:
val example = listOf(
    "..........",
    ".S------7.",
    ".|F----7|.",
    ".||....||.",
    ".||....||.",
    ".|L-7F-J|.",
    ".|..||..|.",
    ".L--JL--J.",
    "..........",
)
getInsideCount(example)

4

## Example 2

In [8]:
val example = listOf(
    ".F----7F7F7F7F-7....",
    ".|F--7||||||||FJ....",
    ".||.FJ||||||||L7....",
    "FJL7L7LJLJ||LJ.L-7..",
    "L--J.L7...LJS7F-7L7.",
    "....F-J..F7FJ|L7L7L7",
    "....L7.F7||L7|.L7L7|",
    ".....|FJLJ|FJ|F7|.LJ",
    "....FJL-7.||.||||...",
    "....L---J.LJ.LJLJ...",
)
getInsideCount(example)

8

## Example 3

In [9]:
val example = listOf(
    "FF7FSF7F7F7F7F7F---7",
    "L|LJ||||||||||||F--J",
    "FL-7LJLJ||||||LJL-77",
    "F--JF--7||LJLJ7F7FJ-",
    "L---JF-JLJ.||-FJLJJ7",
    "|F|F-JF---7F7-L7L|7|",
    "|FFJF7L7F-JF7|JL---7",
    "7-L-JL7||F7|L7F-7F7|",
    "L.L7LFJ|||||FJL7||LJ",
    "L7JLJL-JLJLJL--JLJ.L",
)
getInsideCount(example)

10

## Solution

In [10]:
getInsideCount(input)

285